# Load data

In [534]:
import pandas as pd
import requests
from datetime import datetime

In [535]:
_STOCK = "FPT"

headers = {
        'authority': 'fwtapi2.fialda.com',
        'method': 'GET',
        'scheme' : 'https',
        'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Encoding' : 'gzip, deflate, br',
        'Accept-Language' : 'en-US,en;q=0.9',
        'Cache-Control' : 'max-age=0',
        'Sec-Ch-Ua' : '"Not_A Brand";v="8", "Chromium";v="120", "Microsoft Edge";v="120"',
        'Sec-Ch-Ua-Mobile' : '?0',
        'Sec-Ch-Ua-Platform' : '"Windows"',
        'Sec-Fetch-Dest' : 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site' : 'None',
        'Sec-Fetch-User' : '?1',
        'Upgrade-Insecure-Requests' : '1',
        'User-Agent' : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 Edg/120.0.0.0"
    }
api_url = f"https://fwtapi2.fialda.com/api/services/app/TechnicalAnalysis/GetFinancialHighlights?symbol={_STOCK}"

response = requests.get(api_url,headers=headers)

# Kiểm tra xem có lỗi không (status code 200 là thành công)
if response.status_code == 200:
    # In nội dung phản hồi
    response = response.json()["result"]
else:
    # In lỗi nếu có
    print(f"Lỗi {response.status_code}: {response.text}")

response_lenght = len(response)
df_api = pd.DataFrame()
for i in range(response_lenght):
    df_api = pd.concat([df_api, pd.DataFrame([response[i]])])

df_api = df_api[df_api["quarter"] != 5]
df_api = df_api[::-1]

C:\Users\quang\AppData\Local\Temp\ipykernel_23328\1443263673.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_api = pd.concat([df_api, pd.DataFrame([response[i]])])


In [536]:
current_year = datetime.now().year
lastyear = current_year - 1

temp = df_api[["year", "quarter", "profit"]]
last_year_profit = temp[temp["year"] == lastyear].sum().profit

In [537]:
# CONSTANT
oneYearRatio = 0.3
threeYearRatio = 0.4
fiveYearRatio = 0.3

YEAR_ONE = 0
YEAR_THREE = 2
YEAR_FIVE = 4

THAM_CHIEU_SALEGR = 0.2
TY_TRONG_SALEGR = 0.15

THAM_CHIEU_EPS = 0.2
TY_TRONG_EPS = 0.2

THAM_CHIEU_BVPS = 0.15
TY_TRONG_BVPS = 0.05

THAM_CHIEU_OPC = 0.15
TY_TRONG_OPC = 0.15

THAM_CHIEU_NO_DAI_HAN = last_year_profit * 3
TY_TRONG_NO_DAI_HAN = 0.1

THAM_CHIEU_EFFECTIVENESS = 0.1
TY_TRONG_EFFECTIVENESS = 0.05

THAM_CHIEU_EFFICIENCY = 0.1
TY_TRONG_EFFICIENCY = 0.05

THAM_CHIEU_PRODUCTIVITY = 0.1
TY_TRONG_PRODUCTIVITY = 0.05

THAM_CHIEU_ROA = 0.15
TY_TRONG_ROA = 0.1

THAM_CHIEU_ROE = 0.2
TY_TRONG_ROE = 0.05

THAM_CHIEU_ROIC = 0.15
TY_TRONG_ROIC = 0.15

In [538]:
# NECESSARY
df_api["TOTAL_ASSETS"] = df_api["marketcap"] * df_api["pb"]
df_api["VON_CHU_SO_HUU"] = df_api["profit"] / df_api["mE_ROE"]

In [539]:
def cal_growth_rate(series):
    before = series.shift(-4).values
    after = series.values
    return (after - before)/before

In [540]:
# 1.Sale Growth Rate
Sale = df_api["netSale"]
SaleGR = cal_growth_rate(Sale)

In [541]:
# 2. EPS Growth Rate
eps = df_api["eps"]
epsGR = cal_growth_rate(eps)

In [542]:
# 3. BVPS Growht Rate
eps = df_api["eps"]
socp = df_api["profit"] * 1.0 /eps
bvps = df_api["VON_CHU_SO_HUU"] / socp

In [543]:
bvpsGR = cal_growth_rate(bvps)

In [544]:
# 4.OPC LCDTKD
OPC_LCDTKD = df_api["cF_Operating"]
OPC_LCDTKDGR = cal_growth_rate(OPC_LCDTKD)

In [545]:
# 5. Nợ dài hạn
df_api["NO_DAI_HAN"] = df_api["fS_DebtOnEquityRatio"] * df_api["VON_CHU_SO_HUU"]
temp = df_api[["year", "quarter", "NO_DAI_HAN"]]
no_dai_han = temp[temp["year"] == lastyear].sum().NO_DAI_HAN
no_dai_han

22319018246401.902

In [546]:
# 6. Effectiveness
effectiveness = df_api["netSale"] / df_api["TOTAL_ASSETS"]
effectivenessGR = cal_growth_rate(effectiveness)

In [547]:
# 7. Efficiency
efficiency = df_api["profit"] / df_api["netSale"]
efficiencyGR = cal_growth_rate(efficiency)

In [548]:
# 8.productivity
productivity = OPC_LCDTKD / df_api["profit"]
productivityGR = cal_growth_rate(productivity)

In [549]:
# 9. ROA
ROA = df_api["mE_ROA"]
ROAGR = cal_growth_rate(ROA)

In [550]:
# 10. ROE
ROE = df_api["mE_ROE"]
ROEGR = cal_growth_rate(ROE)

In [551]:
# 11. ROIC
ROIC = df_api["profit"] / (df_api["TOTAL_ASSETS"] + no_dai_han)
ROIC_GR = cal_growth_rate(ROIC)

In [552]:
def calculate_point_item(chiso, tham_chieu):
    val1 = oneYearRatio if chiso[YEAR_ONE] >= tham_chieu else chiso[YEAR_ONE] / tham_chieu * oneYearRatio
    val2 = threeYearRatio if chiso[YEAR_THREE] >= tham_chieu else chiso[YEAR_THREE] / tham_chieu * threeYearRatio
    val3 = fiveYearRatio if chiso[YEAR_FIVE] >= tham_chieu else chiso[YEAR_FIVE] / tham_chieu * fiveYearRatio
    sum_val =  (val1 + val2 + val3) * 100
    if sum_val < 0: return 0
    return sum_val

In [553]:
def calculate4M():
    sum1 = calculate_point_item(SaleGR, THAM_CHIEU_SALEGR)
    sum2 = calculate_point_item(epsGR, THAM_CHIEU_EPS)
    sum3 = calculate_point_item(bvpsGR, THAM_CHIEU_BVPS)
    sum4 = calculate_point_item(OPC_LCDTKDGR, THAM_CHIEU_OPC)
    sum5 = oneYearRatio if no_dai_han >= THAM_CHIEU_NO_DAI_HAN else no_dai_han / THAM_CHIEU_NO_DAI_HAN * oneYearRatio
    sum6 = calculate_point_item(effectivenessGR, THAM_CHIEU_EFFECTIVENESS)
    sum7 = calculate_point_item(efficiencyGR, THAM_CHIEU_EFFICIENCY)
    sum8 = calculate_point_item(productivityGR, THAM_CHIEU_PRODUCTIVITY)
    sum9 = calculate_point_item(ROAGR, THAM_CHIEU_ROA)
    sum10 = calculate_point_item(ROEGR,THAM_CHIEU_ROE)
    sum11 = calculate_point_item(ROIC_GR, THAM_CHIEU_ROIC)

    sum12 = (sum6 + sum7 + sum8) 
    
    return sum1 * TY_TRONG_SALEGR + sum2 * TY_TRONG_EPS + sum3 * TY_TRONG_BVPS + sum4 * TY_TRONG_OPC + sum5 * TY_TRONG_NO_DAI_HAN + sum12 * TY_TRONG_EFFECTIVENESS + sum9 * TY_TRONG_ROA + sum10 * TY_TRONG_ROE + sum11 * TY_TRONG_ROIC

In [554]:
VALUE_4m_BSR = calculate4M()
VALUE_4m_BSR

30.09318090478496